# `boxplots`
This page contains instructions and documentation for creating curve boxplots used to summarize curve ensembles.  

## `functional_boxplot`

A functional boxplot uses curve-based statistics that treat entire curves as a single data point, as opposed to each observation in a curve. 

<img src="../images/sample_func_box_plot.png" alt="Sample Functional Boxplot Plot" width="1400" style="margin:auto;"/>

## `fixed_time_boxplot`

A fixted-time boxplot uses fixed-time statistics that rank each point at each time step, and use those to construct confidence intervals for each time step. 

<img src="../images/sample_ft_box_plot.png" alt="Sample Functional Boxplot Plot" width="1400" style="margin:auto;"/>